In [4]:
!pip install transformers tensorflow datasets kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 77.8 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105786 sha256=80ccc236e00f1c706821c91fe9c23d8777e9277d76dfe21cd0cae70ea5aabeb3
  Stored in directory: /tmp/pip-ephem-wheel-cache-13_vra4d/wheels/2b/af/a9/70bffa2773af622d2ebea9c8d407720b86e67bd40c465bf837
Successfully built kaggle

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
!if [ ! -e ~/.kaggle/kaggle.json ]; then mkdir -p ~/.kaggle; cp ~/kaggle.json ~/.kaggle; echo "done"; fi
!chmod 600 ~/.kaggle/kaggle.json
!ls -alR ~/.kaggle

/opt/app-root/src/.kaggle:
total 12
drwxr-sr-x.  2 1006090000 1006090000 4096 Oct  5 05:34 .
drwxrwsr-x. 18 root       1006090000 4096 Oct  5 07:04 ..
-rw-------.  1 1006090000 1006090000   69 Oct  5 05:34 kaggle.json


In [ ]:
# https://nihcc.app.box.com/v/ChestXray-NIHCC

#kaggle datasets list

!kaggle datasets download paultimothymooney/chest-xray-pneumonia -p chest_xray
!unzip -q chest_xray/*.zip 
!rm -rf chest_xray/__MACOSX

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100%|██████████████████████████████████████▉| 2.29G/2.29G [00:16<00:00, 114MB/s]
100%|███████████████████████████████████████| 2.29G/2.29G [00:16<00:00, 151MB/s]


In [ ]:
import tensorflow as tf

base = tf.keras.applications.VGG19(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(228,228,3),
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

print('VGG19 Loaded')
print(base.summary())

In [ ]:
for pic_file in  np.random.choice(os.listdir(), 3):
    img = mpimg.imread(pic_file)
    plt.imshow(img)
    plt.show()

In [ ]:
# Get model to output 3D feature maps (height, width, features)
altmodel = tf.keras.models.Sequential()

# Layer 1
altmodel.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape=(228, 228, 3)))
altmodel.add(tf.keras.layers.Activation('relu'))
altmodel.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 2
altmodel.add(tf.keras.layers.Conv2D(32, (3, 3)))
altmodel.add(tf.keras.layers.Activation('relu'))
altmodel.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 3
altmodel.add(tf.keras.layers.Conv2D(64, (3, 3)))
altmodel.add(tf.keras.layers.Activation('relu'))
altmodel.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Apply flattening function to convert 3D feature maps in to 1D feature vectors
altmodel.add(tf.keras.layers.Flatten())  

# Add 2 final dense layers to add a classifier to the convolutional base
altmodel.add(tf.keras.layers.Dense(64))
altmodel.add(tf.keras.layers.Activation('relu'))
altmodel.add(tf.keras.layers.Dropout(0.5))
altmodel.add(tf.keras.layers.Dense(1))
altmodel.add(tf.keras.layers.Activation('sigmoid'))

# Compile the model
altmodel.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Print model summary
print(altmodel.summary())


In [39]:
import matplotlib

# Specify dataset size
batch_size = 16

# Specify reference for directory paths
dataset_root = "chest_xray"
classifications = [ "NORMAL", "PNEUMONIA" ]
folders = [ "val", "train", "test" ]

#def preprocess_data(examples):
def dataset_size(directory, labels, showSample=False):
    size = 0 
    for label in labels:
        path = "{}/{}".format(directory, label)
        print("Listing {}".format(path))
        contents = os.listdir(path)
        size += len(contents)
        if showSample:
            for pic_file in  np.random.choice(contents, 2):
                print("Sample {}".format(pic_file))
                img = matplotlib.image.imread("{}/{}".format(path,pic_file))
                plt.imshow(img)
                plt.show()
    return size    

def extract_features(directory, sample_amount):
    features = np.zeros(shape=(sample_amount, 7, 7, 512)) 
    labels = np.zeros(shape=(sample_amount))
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255) 
    generator = datagen.flow_from_directory(
        directory, target_size=(228, 228), 
        batch_size = batch_size, 
        class_mode='binary')
    
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch 
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i = i + 1
        if i * batch_size >= sample_amount:
            break
    return features, labels

# Apply extraction function to 3 datasets
dataset = {}
dataset['features'] = {}
dataset['labels'] = {}

for folder in folders:
    path = "{}/{}".format(dataset_root, folder)
    print("Processing {}...".format(path))
    num_samples = dataset_size(path, classifications, True)
    features, labels = extract_features(path, min(1000, num_samples))
    dataset['features'][folder] = features
    dataset['labels'][folder] = labels

# Shape data
reshape_y = 7 * 7 * 512
for folder in folders:
    print("Reprocessing {}...".format(folder))
    dataset['features'][folder] = np.reshape(dataset['features'][folder], (len(dataset['features'][folder]), reshape_y))

Processing chest_xray/val...
Listing chest_xray/val/NORMAL
Sample NORMAL2-IM-1440-0001.jpeg


FileNotFoundError: [Errno 2] No such file or directory: '/opt/app-root/src/NORMAL2-IM-1440-0001.jpeg'

In [25]:
import re
import os
import numpy as np
import pandas as pd
import tensorflow as tf
#from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    
print(tf.__version__)

Number of replicas: 1
2.15.1


In [ ]:
# Build classifier on top of  VGG19
model = tf.keras.models.Sequential()

# Add dense layers on top of VGG19 
# 1
model.add(tf.keras.layers.Dense(256, activation='relu', input_dim=reshape_y))
# 2
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile
model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=20,
                    batch_size=16,
                    validation_data=(validation_features, validation_labels))

# Save VGG19 results
model.save('models/model_VGG_01.h5')